In [1]:
import pandas as pd

In [3]:
df = pd.read_csv('shop.csv')

In [4]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 139997 entries, 0 to 139996
Data columns (total 9 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   ДатаПродажи        139997 non-null  object
 1   НомерЧека          139997 non-null  int64 
 2   Номенклатура       139997 non-null  object
 3   Штрихкод           139997 non-null  int64 
 4   ТоварнаяГруппа     139997 non-null  object
 5   СН_Характеристика  139996 non-null  object
 6   бгсВес             139997 non-null  object
 7   ОбщаяСумма         139997 non-null  object
 8   Себестоимость      139997 non-null  object
dtypes: int64(2), object(7)
memory usage: 9.6+ MB
None


In [5]:
print(df.head())

    ДатаПродажи  НомерЧека                              Номенклатура  \
0  4/1/23 10:05       1814          серьги с брил.+изумрудом с родир   
1  4/1/23 10:08       4868                         Серьги с топазом    
2  4/1/23 10:08        617  Кольцо с брилл.+изум.гидротерм. с родир.   
3  4/1/23 10:11        461                       Кольцо с амет. род.   
4  4/1/23 10:12       3939                     Цепь Якорь с а/г род.   

        Штрихкод  ТоварнаяГруппа     СН_Характеристика бгсВес ОбщаяСумма  \
0  1078601904686       ДК СЕРЬГИ             Au 585º -   2,89      51180   
1  1078551306013      ПДК СЕРЬГИ             Au 585º -   3,99      34740   
2  1078509166518       ДК КОЛЬЦА  Au 585º -, разм.18,5   2,05      17760   
3  1078579406088  СИ КОЛЬЦО ЦВ/К  Ag 925º -, разм.19,0   1,46        756   
4  1078601866814         СИ ЦЕПЬ  Ag 925º -, разм.40,0    2,7       1560   

  Себестоимость  
0    17276,0475  
1     12074,747  
2      6629,612  
3        302,22  
4      458,9995  


In [6]:
df = pd.read_csv('shop.csv', parse_dates=["ДатаПродажи"]) #формат даты в файле не подходит для работы с ним

In [8]:
df['ДатаПродажи'] = df['ДатаПродажи'].dt.floor("D") #убираю время из даты
df['ДатаПродажи'] = df['ДатаПродажи'].dt.strftime("%m%d%Y") #убираю -
df['ДатаПродажи'] = df['ДатаПродажи'].astype(str).astype(int) #перевожу в числовой формат

In [9]:
def tofloat(total):
    '''Для перевода общей суммы в числовой формат без учёта нулевых и отрицательных значений'''
    if total[0] == '0':
        return 0
    elif total[0] == '-':
        return 0
    return round(float(total.replace(',', '.')), 2)

In [10]:
df['ОбщаяСумма'] = df['ОбщаяСумма'].apply(tofloat) 
df['Себестоимость'] = df['Себестоимость'].apply(tofloat)

In [11]:
#фильтрую DataFrame для оставления нужных значений

df = df[df['ОбщаяСумма'] != 0.00]
df = df[df['Себестоимость'] != 0.00]
df = df[df['ДатаПродажи'] <= 4102023]

In [12]:
def check(material):
    '''Преобразование золота и серебра'''
    if material[:2] == 'СИ':
        return 'Серебро'
    return 'Золото'

In [13]:
df['ТоварнаяГруппа'] = df['ТоварнаяГруппа'].apply(check)
df['Наценка'] = df['ОбщаяСумма'] - df['Себестоимость'] #создаю столбец для подсчёта наценки
df = df.groupby(by = 'ТоварнаяГруппа')['Наценка'].mean() #и группирую по ней

In [14]:
print('Средняя наценка на золото:', round(df['Золото'], 2))
print('Средняя наценка на серебро:', round(df['Серебро'], 2))


Средняя наценка на золото: 8682.81
Средняя наценка на серебро: 1304.24
